In [5]:
import numpy as np
import re
import string
from IPython.display import display, HTML
import string

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

# Paramètres du projet

In [6]:
# Taille du vocabulaire utilisé pour le modèle (nombre de mots uniques pris en compte)
VOCAB_SIZE = 10000

# Longueur maximale des séquences traitées par le modèle
MAX_LEN = 80

# Dimension de l'embedding des mots (représentation vectorielle des mots)
EMBEDDING_DIM = 256

# Dimension des clés pour l'attention multi-têtes
KEY_DIM = 256

# Nombre de "têtes" dans l'attention multi-têtes
N_HEADS = 2

# Dimension du réseau feed-forward dans le Transformer
FEED_FORWARD_DIM = 256

# Fraction des données utilisée pour la validation pendant l'entraînement
VALIDATION_SPLIT = 0.2

# Graine aléatoire pour la reproductibilité
SEED = 42

# Si True, charge un modèle pré-entraîné au lieu de former un nouveau modèle
LOAD_MODEL = False

# Taille des lots utilisés pendant l'entraînement
BATCH_SIZE = 32

# Nombre de cycles complets de passage sur l'ensemble de données pendant l'entraînement
EPOCHS = 10


# Load the data

In [7]:
# Charger les poèmes depuis un fichier txt
with open("poeme_jour13.txt", "r", encoding="utf-8") as file:
    poems = [poem.lower() for poem in file.readlines()]



# Visualisation des premiers mots pour vérification
print(poems[:100])  # Affichage des 1000 premiers mots pour vérification

['\n', '\n', '\n', 'il est temps que je me repose;\n', 'je suis terrassé par le sort.\n', "ne me parlez pas d'autre chose\n", "que des ténèbres où l'on dort!\n", '\n', 'que veut-on que je recommence?\n', 'je ne demande désormais\n', 'à la création immense\n', "qu'un peu de silence et de paix!\n", '\n', "pourquoi m'appelez-vous encore?\n", "j'ai fait ma tâche et mon devoir.\n", "qui travaillait avant l'aurore,\n", "peut s'en aller avant le soir.\n", '\n', 'à vingt ans, deuil et solitude!\n', 'mes yeux, baissés vers le gazon,\n', 'perdirent la douce habitude\n', 'de voir ma mère à la maison.\n', '\n', 'elle nous quitta pour la tombe;\n', "et vous savez bien qu'aujourd'hui\n", 'je cherche, en cette nuit qui tombe,\n', "un autre ange qui s'est enfui!\n", '\n', 'vous savez que je désespère,\n', 'que ma force en vain se défend,\n', 'et que je souffre comme père,\n', 'moi qui souffris tant comme enfant!\n', '\n', "mon oeuvre n'est pas terminée,\n", 'dites-vous. comme adam banni,\n', 'je regar

In [8]:
def remove_punctuation_and_newlines(s):
    # Ajouter de l'espace avant chaque signe de ponctuation
    s = re.sub(f"([{string.punctuation}])", r" \1", s)

    # Supprimer les caractères de ponctuation et les retours à la ligne
    s = re.sub(f"[{string.punctuation}\n]", "", s)

    # Supprimer les espaces multiples
    s = re.sub(" +", " ", s)

    return s.strip()



text_data = [remove_punctuation_and_newlines(x) for x in poems]

In [9]:
example_data = text_data[25]
example_data

'je cherche en cette nuit qui tombe'

In [10]:
# Comptez les poèmes
n_poems = len(text_data)
print(f"{n_poems}   lignes")

10971   lignes


# Tokeniser les données

In [11]:
# Afficher une description
example_data = text_data[30]
example_data

'et que je souffre comme père'

In [12]:
# Convertir en un Dataset TensorFlow
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [13]:
# Créer une couche de vectorisation
vectorize_layer = layers.TextVectorization(
    standardize="lower",                   # Standardiser le texte en le mettant en minuscules
    max_tokens=VOCAB_SIZE,                 # Nombre maximal de tokens uniques dans le vocabulaire
    output_mode="int",                     # Mode de sortie où chaque token est représenté par un entier unique
    output_sequence_length=MAX_LEN + 1,    # Longueur de sortie pour chaque séquence vectorisée
)

In [14]:
# Adapter la couche aux données d'entraînement
vectorize_layer.adapt(text_ds)

# Récupérer le vocabulaire généré par la couche de vectorisation
vocab = vectorize_layer.get_vocabulary()
vocab

['',
 '[UNK]',
 np.str_('et'),
 np.str_('l'),
 np.str_('de'),
 np.str_('la'),
 np.str_('le'),
 np.str_('les'),
 np.str_('des'),
 np.str_('que'),
 np.str_('dans'),
 np.str_('qui'),
 np.str_('à'),
 np.str_('est'),
 np.str_('en'),
 np.str_('un'),
 np.str_('d'),
 np.str_('je'),
 np.str_('du'),
 np.str_('qu'),
 np.str_('nous'),
 np.str_('au'),
 np.str_('ce'),
 np.str_('sur'),
 np.str_('il'),
 np.str_('vous'),
 np.str_('j'),
 np.str_('s'),
 np.str_('où'),
 np.str_('tu'),
 np.str_('une'),
 np.str_('a'),
 np.str_('on'),
 np.str_('aux'),
 np.str_('comme'),
 np.str_('tout'),
 np.str_('se'),
 np.str_('ne'),
 np.str_('pas'),
 np.str_('n'),
 np.str_('pour'),
 np.str_('mon'),
 np.str_('ces'),
 np.str_('ombre'),
 np.str_('plus'),
 np.str_('c'),
 np.str_('sans'),
 np.str_('son'),
 np.str_('dieu'),
 np.str_('sont'),
 np.str_('homme'),
 np.str_('ai'),
 np.str_('quand'),
 np.str_('nuit'),
 np.str_('me'),
 np.str_('ils'),
 np.str_('elle'),
 np.str_('sa'),
 np.str_('tous'),
 np.str_('ma'),
 np.str_('par'),

In [15]:
# Afficher quelques correspondances entre tokens et mots
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: et
3: l
4: de
5: la
6: le
7: les
8: des
9: que


In [16]:
len(vocab)

8961

In [17]:
# Afficher le même exemple converti en entiers
example_tokenised = vectorize_layer(example_data)

# Afficher la séquence d'entiers résultante
print(example_tokenised.numpy())

[  2   9  17 406  34 214   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]


# Créer l'ensemble d'entraînement

In [18]:
def prepare_inputs(text):
    """
    Prépare les entrées pour l'entraînement en créant des paires de phrases :
    l'une avec le texte original et l'autre avec le texte décalé d'un mot.

    Args:
        text (tf.Tensor): Tensor contenant le texte brut.

    Returns:
        tuple: Deux tensors, l'un avec le texte original (x) et l'autre avec le texte décalé d'un mot (y).
    """

    # Étendre les dimensions du texte pour le traitement
    text = tf.expand_dims(text, -1)

    # Convertir le texte en séquences d'entiers
    tokenized_sentences = vectorize_layer(text)

    # x contient tous les mots sauf le dernier de chaque phrase
    x = tokenized_sentences[:, :-1]

    # y contient tous les mots sauf le premier de chaque phrase, décalant ainsi les séquences d'un mot
    y = tokenized_sentences[:, 1:]

    return x, y

# Appliquer la fonction `prepare_inputs` à l'ensemble de données
train_ds = text_ds.map(prepare_inputs)

In [19]:
example_input_output = train_ds.take(1).get_single_element()

In [20]:
# exemple
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  30,   53,    9,   26,  345,  144,   64,   76, 1444,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [21]:
# Exemple de sortie (décalé d'un token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  53,    9,   26,  345,  144,   64,   76, 1444,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

# Créer la fonction de masquage pour l'attention causale

In [22]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Crée un masque pour l'attention causale.

    Ce masque permet de s'assurer que lors de la prédiction d'un token,
    seul le passé (les tokens précédents) est pris en compte.

    Args:
        batch_size (int): Taille du lot.
        n_dest (int): Nombre de tokens de destination.
        n_src (int): Nombre de tokens source.
        dtype (tf.DType): Type des éléments du masque.

    Returns:
        tf.Tensor: Masque d'attention causale de forme [batch_size, n_dest, n_src].
    """

    # Crée des indices pour les tokens de destination et source
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)

    # Détermine la relation entre les indices de destination et de source
    m = i >= j - n_src + n_dest

    # Convertit le masque booléen en type spécifié
    mask = tf.cast(m, dtype)

    # Redimensionne le masque pour le rendre compatible avec les dimensions attendues
    mask = tf.reshape(mask, [1, n_dest, n_src])

    # Duplique le masque pour tout le lot
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

# Affiche le masque d'attention causale transposé pour une meilleure visualisation
np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

# Créer une couche de bloc Transformer

In [23]:
class TransformerBlock(layers.Layer):
    """
    Couche de bloc Transformer.

    Ce bloc est composé d'une attention multi-têtes suivie d'une
    normalisation de couche et d'un réseau feed-forward.

    Attributes:
        num_heads (int): Nombre de têtes pour l'attention multi-têtes.
        key_dim (int): Dimension de la clé pour l'attention.
        embed_dim (int): Dimension de l'embedding.
        ff_dim (int): Dimension du réseau feed-forward interne.
        dropout_rate (float, optional): Taux de dropout. Par défaut à 0.1.
    """

    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate

        # Initialisation des couches
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        """Passage en avant du bloc Transformer."""

        # Calcul du masque d'attention causale
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )

        # Application de l'attention multi-têtes
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)

        # Application du réseau feed-forward
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        """Retourne la configuration du bloc Transformer."""
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

# Créer l'Embedding de Tokens et de Position

In [24]:
class TokenAndPositionEmbedding(layers.Layer):
    """
    Couche d'embedding pour les tokens et les positions.

    Cette couche génère des embeddings à la fois pour les tokens (mots ou caractères)
    et pour leurs positions respectives dans une séquence. L'ajout des embeddings de
    position est une technique couramment utilisée dans des modèles comme Transformer.

    Attributs:
    - max_len: Longueur maximale de la séquence.
    - vocab_size: Taille du vocabulaire.
    - embed_dim: Dimension de l'embedding.
    - token_emb: Couche d'embedding pour les tokens.
    - pos_emb: Couche d'embedding pour les positions.

    Méthodes:
    - call: Génère les embeddings pour une séquence donnée.
    - get_config: Récupère la configuration de la couche pour la sauvegarde et la restauration.
    """

    def __init__(self, max_len, vocab_size, embed_dim):
        """
        Initialise la couche avec les paramètres donnés.

        Paramètres:
        - max_len (int): Longueur maximale de la séquence.
        - vocab_size (int): Taille du vocabulaire.
        - embed_dim (int): Dimension de l'embedding.
        """
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        # Initialisation de la couche d'embedding pour les tokens
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        # Initialisation de la couche d'embedding pour les positions
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        """
        Génère les embeddings pour une séquence donnée.

        Paramètres:
        - x (Tensor): Séquence d'entrée.

        Retour:
        - Tensor: Embeddings des tokens augmentés des embeddings de position.
        """
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        """
        Récupère la configuration de la couche.

        Cette méthode est utilisée pour la sauvegarde et la restauration de la couche.

        Retour:
        - dict: Dictionnaire contenant la configuration de la couche.
        """
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


# Construire le modèle Transformer

In [25]:
# Création de l'entrée du modèle. C'est un tensor d'entiers (indices des tokens)
# avec une longueur indéfinie (d'où le 'None').
inputs = layers.Input(shape=(None,), dtype=tf.int32)

# Application de la couche d'embedding pour les tokens et les positions.
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)

# Application du bloc Transformer, qui renvoie à la fois la sortie du bloc
# et les scores d'attention.
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)

# La sortie est une densité de probabilités sur l'ensemble du vocabulaire
# (taille VOCAB_SIZE) pour chaque position de la séquence d'entrée.
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)

# Création du modèle GPT en liant les entrées et les sorties définies précédemment.
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])

# Compilation du modèle avec l'optimiseur Adam et une fonction de perte
# pour la classification multi-classes. La deuxième perte est définie
# comme 'None' car nous n'entraînons pas le modèle sur les scores d'attention.
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [26]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │     2,580,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 256),    │       658,688 │
│ (TransformerBlock)              │ (None, 2, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10000)    │     2,570,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,809,168 (22.16 MB)

 Trainable params: 5,809,168 (22.16 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

# Entraîner le Transformer

In [28]:
class TextGenerator(callbacks.Callback):
    """
    Callback pour générer du texte à la fin de chaque époque pendant l'entraînement d'un modèle.

    Attributs:
        index_to_word (list): Liste des mots, indexée par leurs indices.
        word_to_index (dict): Dictionnaire des mots et de leurs indices correspondants.
    """

    def __init__(self, index_to_word, top_k=10):
        """
        Initialise le générateur de texte.

        Args:
            index_to_word (list): Liste des mots, indexée par leurs indices.
            top_k (int, optional): Nombre de meilleurs mots à considérer pour le sampling. Par défaut à 10.
        """
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature=1.0):
        """
        Échantillonne un indice de mot à partir de probabilités données.

        Args:
            probs (list): Liste des probabilités.
            temperature (float): Paramètre pour contrôler le degré d'aléatoire lors de l'échantillonnage.

        Returns:
            int: Indice échantillonné.
            list: Probabilités ajustées.
        """
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature=1.0):
        """
        Génère un texte à partir d'un prompt initial.

        Args:
            start_prompt (str): Prompt initial pour la génération de texte.
            max_tokens (int): Nombre maximal de mots à générer.
            temperature (float): Paramètre pour contrôler le degré d'aléatoire lors de l'échantillonnage.

        Returns:
            list: Liste contenant des informations sur chaque mot généré.
        """
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        """
        Méthode appelée à la fin de chaque époque pendant l'entraînement.

        Args:
            epoch (int): Numéro de l'époque actuelle.
            logs (dict, optional): Dictionnaire des logs d'entraînement.
        """
        self.generate("la vie", max_tokens=80, temperature=1.0)

In [29]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)
tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [30]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/10
342/343 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 1.2850
generated text:
la vie y rit 

343/343 ━━━━━━━━━━━━━━━━━━━━ 27s 52ms/step - loss: 1.2813
Epoch 2/10
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.3758
generated text:
la vie tomba les ténèbres l heure hélas 

343/343 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - loss: 0.3758
Epoch 3/10
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.3063
generated text:
la vie et le facteur enfuis ou de tout le jardin 

343/343 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - loss: 0.3064
Epoch 4/10
342/343 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2829
generated text:
la vie est sombre est la muraille 

343/343 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - loss: 0.2829
Epoch 5/10
342/343 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2388
generated text:
la vie est bientôt soûle 

343/343 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - loss: 0.2389
Epoch 6/10
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2072
generated text:
la vie de taupe au lynx 

343/343 ━━━

In [33]:
# Créer le dossier s’il n’existe pas
import os
os.makedirs("models", exist_ok=True)

# Sauvegarder le modèle complet
gpt.save("models/gpt.keras")

In [34]:
def print_probs(info, vocab, top_k=5):
    """
    Affiche le texte avec une mise en évidence basée sur les scores d'attention
    et imprime les probabilités des `top_k` mots les plus probables.

    Paramètres:
    - info (list) : Liste contenant des informations sur le prompt, les scores d'attention et les probabilités des mots.
    - vocab (list) : Liste de mots représentant le vocabulaire.
    - top_k (int) : Nombre de mots les plus probables à afficher.

    """
    # Pour chaque élément dans 'info' (chaque mot généré et ses détails associés)
    for i in info:
        highlighted_text = []

        # Calculer la mise en évidence du texte en fonction des scores d'attention
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                # Crée une mise en évidence basée sur le score d'attention pour le mot actuel
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)

        # Affiche le texte mis en évidence
        display(HTML(highlighted_text))

        # Obtenir les probabilités du mot généré
        word_probs = i["word_probs"]

        # Obtenir les indices et les probabilités des `top_k` mots les plus probables
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]

        # Afficher chaque mot et sa probabilité associée
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")

        # Imprimer une ligne de séparation
        print("--------\n")

In [35]:
info = text_generator.generate(
    "le ciel", max_tokens=1000, temperature=0.5
)


generated text:
le ciel s en va l ombre et qui pleure 



In [36]:
info = text_generator.generate(
    "Un matin", max_tokens=2000, temperature=0.5
)


generated text:
Un matin je me promène 



In [37]:
info = text_generator.generate(
    "la", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


generated text:
la chambre est veuve 



mouche:   	16.579999923706055%
ronce:   	10.550000190734863%
nature:   	9.4399995803833%
rumeur:   	9.40999984741211%
boulangère:   	5.929999828338623%
--------



est:   	99.95999908447266%
a:   	0.029999999329447746%
était:   	0.0%
peut:   	0.0%
au:   	0.0%
--------



veuve:   	100.0%
goutte:   	0.0%
mort:   	0.0%
étroit:   	0.0%
lumière:   	0.0%
--------



:   	100.0%
qui:   	0.0%
de:   	0.0%
divin:   	0.0%
ce:   	0.0%
--------

